In [ ]:
import os
import pandas as pd
import numpy as np
import pdfplumber
from sqlalchemy import create_engine
import json
from datetime import datetime, date, timezone

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()   

DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)

In [ ]:
#Unstructured data
pdf_folder = "data/unstructured"

for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        with pdfplumber.open(os.path.join(pdf_folder, pdf_file)) as pdf:
            text = "\n".join(
                page.extract_text() or "" for page in pdf.pages
            )

        df = pd.DataFrame([{
            "file_name": pdf_file,
            "content": text,
            "pages": len(pdf.pages)
        }])

        df.to_sql("pdf", engine, if_exists="append", index=False)
        print(pdf_file," done successfully")

CODE OF BUSINESS PRINCIPLES - unilever.pdf  done successfully


Cannot set gray non-stroke color because /'P190' is an invalid float value
Cannot set gray non-stroke color because /'P449' is an invalid float value
Cannot set gray non-stroke color because /'P562' is an invalid float value
Cannot set gray non-stroke color because /'P604' is an invalid float value


codeofconduct-infosys.pdf  done successfully
corp-governance-report-2024-nestle.pdf  done successfully
corporate-governance-report-infosys.pdf  done successfully
corporate-governance-report-wipro.pdf  done successfully
HR Policy Manual 2023 iima.pdf  done successfully
IIA HR Policy.pdf  done successfully
infosys-ar-24.pdf  done successfully
infosys-ar-25.pdf  done successfully
tata-ar-24.pdf  done successfully


In [12]:
#Structured data
src_engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/sample_enterprise "
)

tables = [
    "customer",
    "department",
    "employee",
    "employee_project",
    "manager",
    "order_items",
    "orders",
    "product",
    "project"
]

for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", src_engine)
    df.to_sql(table, engine, if_exists="replace", index=False)


In [ ]:
#Semi structured data
from sqlalchemy.dialects.mysql import LONGTEXT

CHUNK_SIZE = 10_000
CSV_FOLDER = "data/semi structured"

def handle_nulls(df):
    for col in df.columns:
        col_lower = col.lower()

        # Skip ID columns
        if col_lower == "id" or col_lower.endswith("_id"):
            continue

        # Numeric columns
        if pd.api.types.is_numeric_dtype(df[col]):
            median = df[col].median()
            if not pd.isna(median):
                df[col] = df[col].fillna(median)

        # Datetime columns
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].fillna(pd.NaT)

        # Text / categorical
        else:
            df[col] = df[col].fillna("Unknown")

    return df

for file in os.listdir(CSV_FOLDER):
    if file.endswith(".csv"):

        file_path = os.path.join(CSV_FOLDER, file)
        table_name = file.replace(".csv", "").lower()

        print(f"Processing {file} → Table: {table_name}")

        first_chunk = True

        for chunk in pd.read_csv(file_path, chunksize=CHUNK_SIZE):

            chunk = handle_nulls(chunk)
            chunk = chunk.replace({np.nan: None})

            chunk.to_sql(
                table_name,
                engine,
                if_exists="replace" if first_chunk else "append",
                index=False,dtype={
                col: LONGTEXT()
                for col in chunk.columns
                if chunk[col].dtype == "object"
            }
            )

            first_chunk = False


Processing customer_support_tickets.csv → Table: customer_support_tickets
Processing emails.csv → Table: emails
Processing glassdoor-companies-reviews.csv → Table: glassdoor-companies-reviews
Processing Tata_Motors_Employee_Reviews.csv → Table: tata_motors_employee_reviews


In [ ]:
#Converting to JSON
def mysql_to_json(
    engine,
    output_file="outputs/ingested.json",
    chunk_size=1000
):
    tables = pd.read_sql("SHOW TABLES", engine).iloc[:, 0].tolist()
    first_record = True
    record_id = 1

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("[\n")

        for table in tables:
            query = f"SELECT * FROM `{table}`"

            for chunk in pd.read_sql(query, engine, chunksize=chunk_size):
                for _, row in chunk.iterrows():
                    row_data = {}

                    for column, value in row.items():
                        # Convert date/datetime to ISO string
                        if isinstance(value, (datetime, date)):
                            row_data[column] = value.isoformat()
                        else:
                            row_data[column] = value

                    record = {
                        "id": record_id,
                        "source_name": table,
                        "timestamp": datetime.now(timezone.utc).isoformat(),
                        "data": row_data
                    }

                    if not first_record:
                        f.write(",\n")
                    else:
                        first_record = False

                    json.dump(record, f, ensure_ascii=False)
                    record_id += 1

            print(f"{table} done")

        f.write("\n]")

    print("MySQL data successfully stored as JSON")